In [2]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder,StandardScaler, Normalizer

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, BatchNormalization

#from keras.layers import Dense,LSTM, Dropout, Input,Conv2D
from keras.models import Sequential

from sklearn.model_selection import train_test_split
epochs = 10


In [3]:
# read data
train_data=pd.read_csv('nsl-kdd/KDDTrain+.txt', sep = ',', error_bad_lines=False, header=None)
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [4]:
encoder = LabelEncoder()
train_data[1]= encoder.fit_transform(train_data.iloc[:,1])
train_data[2]= encoder.fit_transform(train_data.iloc[:,2])
train_data[3]= encoder.fit_transform(train_data.iloc[:,3])
train_ds = train_data.iloc[:,:-2] #Remove the last two columns from train ds

In [5]:
train_labels = train_data.iloc[:,41]
encoded_labels = encoder.fit_transform(train_labels)
train_labels.unique()

array(['normal', 'neptune', 'warezclient', 'ipsweep', 'portsweep',
       'teardrop', 'nmap', 'satan', 'smurf', 'pod', 'back',
       'guess_passwd', 'ftp_write', 'multihop', 'rootkit',
       'buffer_overflow', 'imap', 'warezmaster', 'phf', 'land',
       'loadmodule', 'spy', 'perl'], dtype=object)

In [6]:
#Scale and Normalizer
normalizer = Normalizer()
train_X = normalizer.fit_transform(train_ds)

In [7]:
train_X.shape
train_ds.shape

(125973, 41)

In [8]:
dnn = keras.layers.Input(shape=(train_X.shape[0],))
x = keras.layers.Dense(1024, input_dim=40,activation="relu")(dnn)
x = keras.layers.Dense(1024, activation="relu")(x)
x = keras.layers.Dense(512 ,activation="relu")(x)
#x = keras.layers.BatchNormalization(x)
x = keras.layers.Dense(64,activation="relu")(x)
x = keras.layers.Dense(1,activation="sigmoid")(x)
dnn_model = keras.Model(dnn, x, name="dnn")
dnn_model.summary()

Model: "dnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 125973)]          0         
_________________________________________________________________
dense (Dense)                (None, 1024)              128997376 
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_3 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 130,604,673
Trainable params: 130,604,673
Non-trainable params: 0
_________________________________________________

In [46]:
def print_tensor(x):
    print(x)

In [85]:
class DNNIntrusion(Model):
    
    def __init__(self, data):
        super(DNNIntrusion,self).__init__()
        self.data = data
        self.dense1 = Dense(1024, input_dim=40, activation="relu")
        self.dense2 = Dense(1024,  activation="relu")
        self.dense3 = Dense(1024, activation="relu")
        self.dense4 = Dense(512, activation="relu")
        self.dense5 = Dense(256, activation="relu")
        self.dense6 = Dense(23)
        self.flatten = Flatten()
        self.batch = BatchNormalization()
        
    def call(self, x):
        x = self.dense1(x)
        x = self.batch(x)
        x = self.dense2(x)
        x = self.batch(x)
        x = self.dense3(x)
        x = self.dense4(x)
        #x = self.batch(x)
        x = self.dense5(x)
       # x = self.batch(x)
        print_tensor(x)
        return self.dense6(x)

In [86]:
#Split data
X_train = np.array(train_X)
y_train = np.array(encoded_labels)

In [87]:
cnn_model = DNNIntrusion(train_X)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()


In [88]:
## Metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [89]:
@tf.function
def train_step(images, labels,training=True):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = cnn_model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, cnn_model.trainable_variables)

  optimizer.apply_gradients(zip(gradients, cnn_model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [90]:
EPOCHS = 10
y_train = np.reshape(y_train,(-1,1))

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  train_step(X_train, y_train)

  #for test_images, test_labels in test_ds:
  #  test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}: '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
  )

Tensor("dnn_intrusion_12/dense_81/Relu:0", shape=(125973, 256), dtype=float32)
Tensor("dnn_intrusion_12/dense_81/Relu:0", shape=(125973, 256), dtype=float32)
Epoch 1: Loss: 3.1332859992980957, Accuracy: 0.7588927745819092, 
Epoch 2: Loss: 2.94600772857666, Accuracy: 84.80388641357422, 
Epoch 3: Loss: 2.546586751937866, Accuracy: 84.88247680664062, 
Epoch 4: Loss: 1.8189438581466675, Accuracy: 85.40640258789062, 
Epoch 5: Loss: 1.0810877084732056, Accuracy: 84.06404113769531, 
Epoch 6: Loss: 1.0864123106002808, Accuracy: 84.35061645507812, 
Epoch 7: Loss: 1.234442114830017, Accuracy: 81.9651870727539, 
Epoch 8: Loss: 1.1524783372879028, Accuracy: 84.90629577636719, 
Epoch 9: Loss: 0.9485050439834595, Accuracy: 85.05314636230469, 
Epoch 10: Loss: 0.7381891012191772, Accuracy: 84.86739349365234, 
